In [1]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

load_dotenv(PARENT_DIR / '.env')

True

In [10]:
HOPSWORK_PROJECT_NAME = 'taxi_demand_ytjqdg'
HOPWSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [3]:
import pandas as pd
from datetime import datetime
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f"Downloading raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
2023-03 file is not available
2023-04 file is not available
2023-05 file is not available
2023-06 file is not available
2023-07 file is not available
2023-08 file is not available
2023-09 file is not available
2023-10 file is not available
2023-11 file is not available
2023-12 file is not available


In [4]:
print(f'{len(rides)=}')

len(rides)=45635252


In [5]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:01<00:00, 136.43it/s]


In [6]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,0,1
1,2022-01-01 01:00:00,0,1
2,2022-01-01 02:00:00,0,1
3,2022-01-01 03:00:00,0,1
4,2022-01-01 04:00:00,1,1
...,...,...,...
2696635,2023-02-28 19:00:00,5,265
2696636,2023-02-28 20:00:00,5,265
2696637,2023-02-28 21:00:00,7,265
2696638,2023-02-28 22:00:00,5,265


In [ ]:
print(f'{len(ts_data)=}')

In [8]:
import hopsworks

In [11]:
project = hopsworks.login(
    project=HOPSWORK_PROJECT_NAME,
    api_key_value=HOPWSWORKS_API_KEY
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/47255


In [12]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [13]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [15]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [16]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/47255/fs/46149/fg/49515


Uploading Dataframe: 0.00% |          | Rows 0/2696640 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/47255/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fe4541c4460>, None)